# Flux.Tracker

バックプロパゲーション、またはリバースモード自動微分は、`Flux.Tracker`モジュールで処理される。

In [1]:
using Flux.Tracker

ここではこのモジュールのより高度な使い方とその内部について解説する。

## Taking Gradients
---

[basics session](https://fluxml.ai/Flux.jl/stable/models/basics.html)で`gradient`関数の基本的な使い方をカバーした。

In [2]:
using Flux.Tracker

Tracker.gradient((a,b) -> a*b, 2, 3)

(3.0 (tracked), 2.0 (tracked))

`gradient`は実際には、backpropagatorに基づくインターフェイス周りの薄いラッパーの`forward`のことだ。

In [7]:
using Flux.Tracker: forward

@show y, back = forward((a,b)->a*b, 2, 3)

@show back(1)

(y, back) = forward(((a, b)->begin
                #= In[7]:3 =#
                a * b
            end), 2, 3) = (6.0 (tracked), getfield(Flux.Tracker, Symbol("##9#11")){getfield(Flux.Tracker, Symbol("##6#7")){Params,Flux.Tracker.TrackedReal{Float64}}}(Core.Box((2.0 (tracked), 3.0 (tracked))), getfield(Flux.Tracker, Symbol("##6#7")){Params,Flux.Tracker.TrackedReal{Float64}}(Params([3.0 (tracked), 2.0 (tracked)]), 6.0 (tracked))))
back(1) = (3.0 (tracked), 2.0 (tracked))


(3.0 (tracked), 2.0 (tracked))

`forward`関数は２つの結果を返す。まず`y`は（おそらくトラッキングが適用された場合）、関数のオリジナルの値だ。  
次に`back`は、感度（sensitivity）が与えられて、`forward`に入力の感度（sensitivity）を返す新しい関数だ。（これを`backpropagor`と呼ぶ)  
このインターフェイスの用途の一つは、カスタムの感度を提供することだ、出力がスカラーではないときに。

In [11]:
y, back = forward((a,b) -> a.*b, [1,2,3],[4,5,6])

(Flux.Tracker.TrackedReal{Float64}[4.0 (tracked), 10.0 (tracked), 18.0 (tracked)], getfield(Flux.Tracker, Symbol("##9#11")){getfield(Flux.Tracker, Symbol("##6#7")){Params,TrackedArray{…,Array{Float64,1}}}}(Core.Box((Flux.Tracker.TrackedReal{Float64}[1.0 (tracked), 2.0 (tracked), 3.0 (tracked)], Flux.Tracker.TrackedReal{Float64}[4.0 (tracked), 5.0 (tracked), 6.0 (tracked)])), getfield(Flux.Tracker, Symbol("##6#7")){Params,TrackedArray{…,Array{Float64,1}}}(Params([Flux.Tracker.TrackedReal{Float64}[4.0 (tracked), 5.0 (tracked), 6.0 (tracked)], Flux.Tracker.TrackedReal{Float64}[1.0 (tracked), 2.0 (tracked), 3.0 (tracked)]]), Flux.Tracker.TrackedReal{Float64}[4.0 (tracked), 10.0 (tracked), 18.0 (tracked)])))

In [16]:
back([1,1,1])

(Flux.Tracker.TrackedReal{Float64}[4.0 (tracked), 5.0 (tracked), 6.0 (tracked)], Flux.Tracker.TrackedReal{Float64}[1.0 (tracked), 2.0 (tracked), 3.0 (tracked)])

適当な場所に勾配を取ることもできる。  
これは一階の勾配しか気にしないなら便利だ。

In [18]:
a,b = param(2), param(3)

@show c=a*b

Tracker.back!(c)

@show Tracker.grad(a), Tracker.grad(b)

c = a * b = 6.0 (tracked)
(Tracker.grad(a), Tracker.grad(b)) = (3.0, 2.0)


(3.0, 2.0)

## Tracked Arrays
---

`param`関数は普通のJulia配列を新しいオブジェクトに変換する。  
新しいオブジェクトは配列のように振る舞い、導関数を計算するための追加情報を追跡する。  
例えば、２つのパラメータを掛けるとする。

In [19]:
W = param([1 2; 3 4])

Tracked 2×2 Array{Float64,2}:
 1.0  2.0
 3.0  4.0

In [20]:
x = param([5, 6])

Tracked 2-element Array{Float64,1}:
 5.0
 6.0

In [21]:
y = W * x

Tracked 2-element Array{Float64,1}:
 17.0
 39.0

出力`y`は`TrackedArray`オブジェクトでもある。  
`back!`関数を介して`W`と`b`へ感度をバックプロパゲーションできる様になり、Tracked Array `W`や`x`の累積した勾配が分かる。

In [22]:
Tracker.back!(y, [1,-1])

In [23]:
W.grad

2×2 Array{Float64,2}:
  5.0   6.0
 -5.0  -6.0

In [24]:
x.grad

2-element Array{Float64,1}:
 -2.0
 -2.0

デリバティブ情報を削除して、プレーンな値を得た方が良いかもしれない。  これは`Tracker.data(W)`を呼べばできる。

## Custom Gradients
---

関数やカーネルにカスタム勾配を実装するために、上述のプロセスをフックすることができる。  
ちょっとした例として、`minus`のカスタム実装を想像してみよう。

In [26]:
minus(a,b) = a-b

minus (generic function with 1 method)

まず、minusへの呼び出しを見たときにストップしてそれを記録するためにするために、トラッカーシステムに指示しないといけない。  
これはディスパッチを使えばできる。

In [37]:
using Flux.Tracker: TrackedArray, track, @grad, data

minus(a::TrackedArray, b::TrackedArray) = track(minus, a, b)

minus (generic function with 2 methods)

`track`は新しいTracked objectの作成と、テープ上でのオペレーションの記録を引き受ける。  
勾配の定義を与えないといけない。

In [38]:
@grad function minus(a,b)
    return minus(data(a), data(b)), Δ -> (Δ, -Δ)
end

これは本質的に、単に上で見た`forward`関数をオーバーロードする方法だ。  
`a`と`b`からトラッキングを取り除き、minusのオリジナルの定義を呼び出している。 (そうでない場合、再度呼び出して無限回帰を実行するだろう）

backpropagatorでは`data(a)`を呼び出さないことに注意。  
ネストした自動微分はbackpropagator自体を通してデリバティブを取るから、実際はこれを追跡したい。  
例えば、`*`の勾配はこのようかも。

In [39]:
@grad a*b = data(a)*data(b), Δ ->(Δ*b, a*Δ)

以下のように`minus`の一階微分を計算できる。

In [47]:
a = param([1,2,3])
b = param([3,2,1])

@show c = minus(a,b)

Tracker.back!(c, 1)
@show Tracker.grad(a)
@show Tracker.grad(b)

c = minus(a, b) = Flux.Tracker.TrackedReal{Float64}[-2.0 (tracked), 0.0 (tracked), 2.0 (tracked)]
Tracker.grad(a) = [1.0, 1.0, 1.0]
Tracker.grad(b) = [-1.0, -1.0, -1.0]


3-element Array{Float64,1}:
 -1.0
 -1.0
 -1.0

カスタム勾配を持つ複数引数の関数の場合、単に`minus(::TrackedArray, ::TrackedArray)`だけでなく、`minus(::Array, ::TrackedArray)`などもまた、キャッチする必要がある。  
そのためには、必要に応じてそれらの追加シグネチャを定義するだけだ。

In [48]:
minus(a::AbstractArray, b::TrackedArray) = Tracker.track(minus, a, b)
minus(a::TrackedArray, b::AbstractArray) = Tracker.track(minus, a, b)

minus (generic function with 4 methods)

## Tracked Internals
---

すべての`Tracked*`オブジェクト（`TrackedArray`, `TrackedReal`)は`Tracked`型を囲む軽いラッパーで、`.tracker`フィールドを介してアクセスができる。  

In [49]:
x.tracker

Flux.Tracker.Tracked{Array{Float64,1}}(0x00000000, Flux.Tracker.Call{Nothing,Tuple{}}(nothing, ()), true, [-2.0, -2.0])

`Tracker`は与えられたオブジェクトの勾配を保存する。 これは前に見たね。 // ←この訳でいいのだろうか？

In [50]:
x.tracker.grad

2-element Array{Float64,1}:
 -2.0
 -2.0

trackerは`Call`オブジェクトも含み、これはforwardパス中のある時点で行われた関数呼び出しを単に表す。  
例えば、`+`呼び出しはこのようだ。

In [83]:
Tracker.Call(+, (1,2) ) # 元コードと違う

Flux.Tracker.Call{typeof(+),Tuple{Int64,Int64}}(+, (1, 2))

上で作った`y`の場合、それを生み出した呼び出しを保存していることが分かる,つまり`W*x`のこと。

In [84]:
y.tracker.f

Flux.Tracker.Call{getfield(Flux.Tracker, Symbol("##445#446")){TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}},Tuple{Flux.Tracker.Tracked{Array{Float64,2}},Flux.Tracker.Tracked{Array{Float64,1}}}}(getfield(Flux.Tracker, Symbol("##445#446")){TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}}(Flux.Tracker.TrackedReal{Float64}[1.0 (tracked) 2.0 (tracked); 3.0 (tracked) 4.0 (tracked)], Flux.Tracker.TrackedReal{Float64}[5.0 (tracked), 6.0 (tracked)]), (Flux.Tracker.Tracked{Array{Float64,2}}(0x00000000, Flux.Tracker.Call{Nothing,Tuple{}}(nothing, ()), true, [5.0 6.0; -5.0 -6.0]), Flux.Tracker.Tracked{Array{Float64,1}}(0x00000000, Flux.Tracker.Call{Nothing,Tuple{}}(nothing, ()), true, [-2.0, -2.0])))

呼び出しの引数がTracked Arrayで、自身の呼び出しを保存していることもあるため、これは`Tracker`がforwardパス中に起こったことすべてを記録するデータ構造を構成することを意味する。

`back!(y, [1, -1])`を呼ぶとき、感度`[1, -1]`は単に`y`の呼び出し(`*`)に送られる。

In [92]:
Tracker.back(*, [1, -1], W, x) # ＼(^o^)／

MethodError: MethodError: no method matching back(::typeof(*), ::Array{Int64,1}, ::TrackedArray{…,Array{Float64,2}}, ::TrackedArray{…,Array{Float64,1}})
Closest candidates are:
  back(!Matched::Flux.Tracker.Tracked, ::Any) at /Users/umikoz/.julia/packages/Flux/jsf3Y/src/tracker/back.jl:35
  back(!Matched::Nothing, ::Any) at /Users/umikoz/.julia/packages/Flux/jsf3Y/src/tracker/back.jl:50
  back(!Matched::Flux.Tracker.Grads, !Matched::Flux.Tracker.Tracked, ::Any) at /Users/umikoz/.julia/packages/Flux/jsf3Y/src/tracker/back.jl:112
  ...

これは引数(`W`や`b`)の感度を計算し、呼び出しを通して逆伝搬する。  
これは再帰的だから、プログラムグラフ全体を調べてオリジナルのモデルパラメータへ勾配を伝搬する。